In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

In [2]:
# Load the CSV dataset
df = pd.read_csv('../inputs/kickstarter_projects.csv', 
                 parse_dates=['deadline', 'launched'])

In [3]:
# Drop live projects
df = df.query('state != "live"')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375862 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                375862 non-null  int64         
 1   name              375858 non-null  object        
 2   category          375862 non-null  object        
 3   main_category     375862 non-null  object        
 4   currency          375862 non-null  object        
 5   deadline          375862 non-null  datetime64[ns]
 6   goal              375862 non-null  float64       
 7   launched          375862 non-null  datetime64[ns]
 8   pledged           375862 non-null  float64       
 9   state             375862 non-null  object        
 10  backers           375862 non-null  int64         
 11  country           375862 non-null  object        
 12  usd pledged       372066 non-null  float64       
 13  usd_pledged_real  375862 non-null  float64       
 14  usd_

In [4]:
# Remove corrupted data: when state = "successful" and backers = 0
df = df[~((df['state'] == 'successful') & (df['backers'] == 0))]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375757 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                375757 non-null  int64         
 1   name              375753 non-null  object        
 2   category          375757 non-null  object        
 3   main_category     375757 non-null  object        
 4   currency          375757 non-null  object        
 5   deadline          375757 non-null  datetime64[ns]
 6   goal              375757 non-null  float64       
 7   launched          375757 non-null  datetime64[ns]
 8   pledged           375757 non-null  float64       
 9   state             375757 non-null  object        
 10  backers           375757 non-null  int64         
 11  country           375757 non-null  object        
 12  usd pledged       372066 non-null  float64       
 13  usd_pledged_real  375757 non-null  float64       
 14  usd_

In [5]:
# Drop all rows that contain any null values
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 372062 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                372062 non-null  int64         
 1   name              372062 non-null  object        
 2   category          372062 non-null  object        
 3   main_category     372062 non-null  object        
 4   currency          372062 non-null  object        
 5   deadline          372062 non-null  datetime64[ns]
 6   goal              372062 non-null  float64       
 7   launched          372062 non-null  datetime64[ns]
 8   pledged           372062 non-null  float64       
 9   state             372062 non-null  object        
 10  backers           372062 non-null  int64         
 11  country           372062 non-null  object        
 12  usd pledged       372062 non-null  float64       
 13  usd_pledged_real  372062 non-null  float64       
 14  usd_

In [6]:
# Deal with Dependent Variable State
df['target'] = df['state'].apply(lambda x: 0 if x == 'successful' else 1)
print(df['target'].value_counts())

1    238211
0    133851
Name: target, dtype: int64


In [7]:
# Feature Engineering
df['launched'] = pd.to_datetime(df['launched'])
df['deadline'] = pd.to_datetime(df['deadline'])
df['duration'] = (df['deadline'] - df['launched']).dt.days

In [8]:
# Extract year-month and convert to string
df['launched_year_month'] = df['launched'].dt.to_period('M').astype(str)
df['deadline_year_month'] = df['deadline'].dt.to_period('M').astype(str)

In [9]:
# Convert Datetime columns to categorical columns
date_time_cols = ['launched_year_month', 'deadline_year_month']
df[date_time_cols] = df[date_time_cols].astype('category')

In [10]:
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,target,duration,launched_year_month,deadline_year_month
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:00,0.0,failed,0,GB,0.0,0.0,1533.95,1,58,2015-08,2015-10
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:00,2421.0,failed,15,US,100.0,2421.0,30000.00,1,59,2017-09,2017-11
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:00,220.0,failed,3,US,220.0,220.0,45000.00,1,44,2013-01,2013-02
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:00,1.0,failed,1,US,1.0,1.0,5000.00,1,29,2012-03,2012-04
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:00,1283.0,canceled,14,US,1283.0,1283.0,19500.00,1,55,2015-07,2015-08


In [11]:
cat_features = ['category', 'currency', 
                'country', 'launched_year_month',
               'deadline_year_month']
encoder = LabelEncoder()

encoded = df[cat_features].apply(encoder.fit_transform)
encoded.head()

,category,currency,country,launched_year_month,deadline_year_month
0,108,5,9,76,77
1,93,13,21,101,102
2,93,13,21,45,45
3,90,13,21,35,35
4,55,13,21,75,75


In [12]:
df = df[['backers','usd_pledged_real', 'usd_goal_real',
        'target']].join(encoded)
df.head()

,backers,usd_pledged_real,usd_goal_real,target,category,currency,country,launched_year_month,deadline_year_month
0,0,0.0,1533.95,1,108,5,9,76,77
1,15,2421.0,30000.00,1,93,13,21,101,102
2,3,220.0,45000.00,1,93,13,21,45,45
3,1,1.0,5000.00,1,90,13,21,35,35
4,14,1283.0,19500.00,1,55,13,21,75,75


In [13]:
# Define target and features
X = df.drop(['target'
              ], axis=1)
y = df['target']

## Training a LightGBM model
We will use a LightGBM model. This is a tree-based model that typically provides the best performance, even compared to XGBoost. It's also relatively fast to train.

In [14]:
!pip install lightgbm

In [15]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

# Define the hyperparameter grid for grid search
param_grid = {
    'num_leaves': [4],
    'objective': ['binary'],
    'metric': ['auc'],
}

In [16]:
# Create the LightGBM model
lgb_model = lgb.LGBMClassifier()

# Create the GridSearchCV object
grid_search = GridSearchCV(lgb_model, param_grid, cv=3, scoring='roc_auc')

# Fit the GridSearchCV on the training data
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

# Create the final LightGBM model with the best hyperparameters
final_lgb_model = lgb.LGBMClassifier(**best_params)

# Fit the final model on the training data
final_lgb_model.fit(X_train, y_train)

# Make predictions on the test data using the final model
y_pred_lgb = final_lgb_model.predict(X_test)

# Evaluate the final model
print("Classification Report:")
print(classification_report(y_test, y_pred_lgb))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_lgb))

[LightGBM] [Info] Number of positive: 111228, number of negative: 62400
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003256 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1172
[LightGBM] [Info] Number of data points in the train set: 173628, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.640611 -> initscore=0.578017
[LightGBM] [Info] Start training from score 0.578017
[LightGBM] [Info] Number of positive: 111228, number of negative: 62401
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002952 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1172
[LightGBM] [Info] Number of data points in the train set: 173629, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.640607 -> initscore=0.5780

In [20]:
# Get feature importances from the final model
importances = final_lgb_model.feature_importances_
print("Feature importances:")
for col, imp in zip(X_train.columns, importances):
    print(f"{col}: {imp}")

Feature importances:
backers: 21
usd_pledged_real: 146
usd_goal_real: 133
category: 0
currency: 0
country: 0
launched_year_month: 0
deadline_year_month: 0


In [25]:
# Get AUC score on the test data
ypred_prob = final_lgb_model.predict_proba(X_test)[:, 1]
score = roc_auc_score(y_test, ypred_prob)
print(f"Test AUC score: {score}")

Test AUC score: 0.9974744178383903


In [29]:
# Separate successful and unsuccessful projects
successful_projects = df[df['target'] == 0]
unsuccessful_projects = df[df['target'] == 1]

# Calculate statistical significance for each numerical feature using t-test
print("Statistical significance of numerical features:")
significant_features = []
for col in df.drop('target', axis=1).columns:
    if col not in ['category', 'currency', 'country']:
        t_stat, p_value = ttest_ind(successful_projects[col], unsuccessful_projects[col], equal_var=False)
        print(f"{col}: t-statistic = {t_stat:.4f}, p-value = {p_value:.4f}")
        if p_value < 0.05:
            significant_features.append(col)

print("\nSignificant features with p-value < 0.05:")
print(significant_features)

Statistical significance of numerical features:
backers: t-statistic = 59.5760, p-value = 0.0000
usd_pledged_real: t-statistic = 51.0818, p-value = 0.0000
usd_goal_real: t-statistic = -19.1745, p-value = 0.0000
launched_year_month: t-statistic = -51.0789, p-value = 0.0000
deadline_year_month: t-statistic = -52.4675, p-value = 0.0000

Significant features with p-value < 0.05:
['backers', 'usd_pledged_real', 'usd_goal_real', 'launched_year_month', 'deadline_year_month']
